In [1]:
import sys

print("Используем Python {}".format(sys.version))

Используем Python 3.6.3 (default, Oct  3 2017, 21:45:48) 
[GCC 7.2.0]


In [2]:
import pandas as pd

print("Используем pandas {}".format(pd.__version__))

Используем pandas 0.20.3


In [3]:
import scipy

print("Используем scipy {}".format(scipy.__version__))

Используем scipy 1.0.0


In [4]:
import numpy as np

print("Используем numpy {}".format(np.__version__))

Используем numpy 1.13.3


In [5]:
# эта функция нам поможет считывать данные
from os import path

def read_csv(filename: str):
    data_dir = "../../data/ml-latest-small"
    data = pd.read_csv(path.join(data_dir, filename + ".csv"))
    return data

ratings = read_csv("ratings")
movies = read_csv("movies")

In [6]:
# пользователи ставят фильмам оценки
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
# про фильмы знаем названия и жанры
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
# максимальный ID фильма
last_movie_id = ratings["movieId"].max()
print(last_movie_id)
# максимальный ID пользователя
last_user_id = ratings["userId"].max()
print(last_user_id)

163949
671


In [9]:
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix):
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )

user_x_item = ratings[["userId", "movieId"]].as_matrix()
user_x_item
# здесь уже на пересечении строк и столбцов матрицы стоят не единички, а реальные оценки
user_item_matrix = csr_matrix(
    (
        ratings["rating"].tolist(),
        (
            [pair[0] for pair in user_x_item],
            [pair[1] for pair in user_x_item],
        )
    ),
    shape=(last_user_id + 1, last_movie_id + 1),
    dtype=np.float32
)

sparse_info(user_item_matrix)

Размерности матрицы: (672, 163950)
Ненулевых элементов в матрице: 100004
Доля ненулевых элементов: 0.0009076881743853382


In [10]:
# представление фильма в виде вектора его оценок
movie_id = 260
print(user_item_matrix[:, movie_id])

  (4, 0)	5.0
  (7, 0)	5.0
  (8, 0)	3.5
  (15, 0)	5.0
  (17, 0)	3.5
  (18, 0)	3.0
  (19, 0)	4.0
  (20, 0)	1.5
  (21, 0)	3.0
  (22, 0)	4.0
  (23, 0)	4.5
  (25, 0)	4.0
  (26, 0)	3.0
  (30, 0)	4.0
  (31, 0)	4.0
  (34, 0)	4.0
  (36, 0)	5.0
  (40, 0)	4.5
  (41, 0)	4.0
  (42, 0)	4.5
  (44, 0)	3.0
  (55, 0)	5.0
  (56, 0)	2.0
  (57, 0)	4.0
  (62, 0)	4.0
  :	:
  (607, 0)	4.5
  (611, 0)	2.0
  (613, 0)	5.0
  (614, 0)	4.0
  (615, 0)	3.5
  (616, 0)	5.0
  (620, 0)	4.0
  (622, 0)	5.0
  (624, 0)	5.0
  (626, 0)	3.0
  (627, 0)	4.0
  (636, 0)	3.0
  (640, 0)	5.0
  (643, 0)	5.0
  (646, 0)	5.0
  (647, 0)	5.0
  (651, 0)	5.0
  (654, 0)	5.0
  (655, 0)	2.5
  (659, 0)	5.0
  (660, 0)	4.5
  (664, 0)	4.5
  (665, 0)	4.0
  (669, 0)	5.0
  (671, 0)	5.0


In [11]:
# теперь найдём матрицу схожести между фильмами
from sklearn.preprocessing import normalize

def get_cosine_similarity_matrix(user_item_matrix):
    # матрицы этого типа быстрее умножаются
    user_item_csr = user_item_matrix.tocsr()
    # нормализация и последующее умножение эквивалентно нахождению косинуса между столбцами матрицы
    # user_item_normalized = normalize(user_item_csr, norm='l1', axis=0)
    user_item_normalized = user_item_csr
    return user_item_normalized.T.dot(user_item_normalized)

similarity_matrix = get_cosine_similarity_matrix(user_item_matrix)
sparse_info(similarity_matrix)

Размерности матрицы: (163950, 163950)
Ненулевых элементов в матрице: 21983224
Доля ненулевых элементов: 0.0008178403679890727


In [12]:
from scipy.sparse import vstack, csr_matrix

k = 5

# оставляем только top k схожих элементов (k ближайших соседей)
def get_top_k_in_a_row(similarity_matrix, row, k, no_diagonal):
    lil_row = similarity_matrix[row]
    # каждый фильм очень сильно схож сам с собой, но это бесполезная информация
    if no_diagonal:
        lil_row[0, row] = 0
    csr_row = lil_row.tocsr()
    csr_row.data[csr_row.data.argsort()[:-k]] = 0
    csr_row.eliminate_zeros()
    return csr_row

def get_top_k(similarity_matrix, k, no_diagonal=True):
    # с матрицами этого типа удобнее всего работать построчно
    lil_similarity = similarity_matrix.tolil()
    top_k_similarity_matrix = get_top_k_in_a_row(lil_similarity, 0, k, no_diagonal)
    for row in range(1, lil_similarity.shape[0]):
        if len(lil_similarity.rows[row]) > 0:
            csr_row = get_top_k_in_a_row(lil_similarity, row, k, no_diagonal)
            top_k_similarity_matrix = vstack([top_k_similarity_matrix, csr_row])
    return top_k_similarity_matrix

top_k_similarity_matrix = get_top_k(similarity_matrix, k)

In [13]:
# ненулевых элементов стало гораздо меньше
print("Было:", similarity_matrix.nnz)
print("Стало:", top_k_similarity_matrix.nnz)

Было: 21983224
Стало: 45330


In [14]:
# выделяем обучающую и тестовую выборки
from scipy.sparse import coo_matrix

np.random.seed(0)
train_percent = 0.8
user_item_matrix = user_item_matrix.tocoo()
train_split = np.random.choice(
    range(user_item_matrix.nnz),
    int(user_item_matrix.nnz * train_percent),
    replace=False
)
test_split = list(set(range(user_item_matrix.nnz)) - set(train_split))
train_matrix = coo_matrix(
    (
        user_item_matrix.data[train_split],
        (user_item_matrix.row[train_split], user_item_matrix.col[train_split])
    ),
    shape=user_item_matrix.shape
)
test_matrix = coo_matrix(
    (
        user_item_matrix.data[test_split],
        (user_item_matrix.row[test_split], user_item_matrix.col[test_split])
    ),
    shape=user_item_matrix.shape
)
print("Размер обучающей выборки:", train_matrix.nnz)
print("Размер тестовой выборки:", test_matrix.nnz)

Размер обучающей выборки: 80003
Размер тестовой выборки: 20001


In [15]:
# нахождение предсказаний эквивалентно умножению нормированной матрицы схожести на вектор оценок пользователя
normalized_similarity = normalize(similarity_matrix.tocsr(), norm="l1", axis=1)
raw_predictions = train_matrix.dot(normalized_similarity)
# но мы не можем рекомендовать каждому пользователю больше N элементов
N = 20
recommendations = get_top_k(raw_predictions, N, no_diagonal=False)

In [16]:
# какая-то оценка качества
MSE = sum((user_item_matrix - test_matrix).data ** 2)
print(MSE)

1093264.25
